In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import math
import pickle
import os
import sys

In [ ]:
sys.path.append("../")
print(os.getcwd())
from local_config import LocalConfig

config = LocalConfig.setup(path_to_root="../") #root path depends on file hierarchy


In [ ]:
# Reload config files
config.load_config()

In [ ]:

path_to_embedding = config.getPath("old_project") + config.getPath("path_embedding.pkl")
path_to_vectors = config.getPath("old_project") + config.getPath("animation_vectors.pkl")
path_to_path_selector_label = config.getPath("old_project") + config.getPath("path_selector_label.pkl")
print(path_to_embedding, path_to_vectors, path_to_path_selector_label)
print(os.path.exists(path_to_embedding))
print(os.path.exists(path_to_vectors))
print(os.path.exists(path_to_path_selector_label))

In [ ]:
with open(path_to_path_selector_label, "rb") as f:
    inp = pickle.load(f)
inp

In [ ]:
with open(path_to_embedding, "rb") as f:
    inp = pickle.load(f)

inp["filename"] = inp["filename"] + "_animation_0"
inp

In [ ]:
with open(path_to_vectors, "rb") as f:
    output = pickle.load(f)


output = pd.merge(inp, output, left_on="filename", right_on="file", how="inner").drop_duplicates("filename")
output = output.drop(["filename", "animation_id_x", "file", "animation_id_y", "order_id", "path_prob", "begin_value", "animated_animation_ids", "animated_order_ids", "backend_mapping"], axis=1)
modelOutput = pd.DataFrame(output["model_output"].to_list(), columns=["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"])

output["index"] = range(1, len(output) + 1)
modelOutput["index"] = range(1, len(modelOutput) + 1)

output.reset_index(drop=True)
modelOutput.reset_index(drop=True)
output = pd.merge(output, modelOutput,how="inner", on="index")

output = output.drop(["model_output", "index"], axis=1)

input = output.copy()

input[["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"]] = np.float32(0)

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 300)

In [ ]:
inpTensor = torch.tensor(list(input.values))
rows, cols = inpTensor.shape
inpTensor = inpTensor.view(1, rows, cols)
print(inpTensor)
print(inpTensor.shape)
print(inpTensor.dtype)

torch.set_printoptions(threshold=1000)
outTensor = torch.tensor(list(output.astype(np.float32).values))
rows, cols = outTensor.shape
outTensor = outTensor.view(1, rows, cols)
print(outTensor)
print(outTensor.shape)
print(outTensor.dtype)

test = inpTensor[:, :1, :]
print(test)
print(test.shape)

In [ ]:
with open(path_to_embedding, "rb") as f:
    inp = pickle.load(f)

#print(inp)

with open(path_to_vectors, "rb") as f:
    output = pickle.load(f)
output = output[output["label"].isin(["Good", "Very Good"])][["file","animation_id","model_output","label"]]

logos = output["file"].str.split("_animation").str[0]

bestOutput = pd.DataFrame()
for logo in logos:
    temp = output[output["file"].str.contains(logo)]
    if temp["label"].eq("Very Good").any():
        bestOutput = pd.concat([bestOutput, pd.DataFrame(temp[temp["label"] == "Very Good"].iloc[0])], axis=1, ignore_index=True)
    else:
        bestOutput = pd.concat([bestOutput, pd.DataFrame(temp.iloc[0])], axis=1, ignore_index=True)
bestOutput=bestOutput.transpose()
print(bestOutput)


names = output["file"].str.replace("_animation_0", "")

inp = inp[inp["filename"].isin(names)]

filenames = inp["filename"].unique()
list = []
for name in filenames:
    seq = inp[inp["filename"]==name].loc[:, ~inp.columns.isin(["filename","animation_id"])][:4]
    seq = pd.concat([seq, pd.DataFrame(0, index=seq.index, columns=range(256, 268))], axis=1, ignore_index=True)
    
    sos = pd.DataFrame([[10]*268])
    
    eos = pd.DataFrame([[20]*268])
    
    seq = pd.concat([sos, seq, eos], ignore_index=True)

    list.append(torch.tensor(seq.values))
inpTensor = torch.stack(list)
print(inpTensor.shape)

In [ ]:
with open(path_to_vectors, "rb") as f:
    output = pickle.load(f)
model_output = output[output['file'].str.contains('animation_0')][["file","model_output"]]

filenames = model_output["file"].unique()
list = []
for name in filenames:
    seq = model_output[model_output["file"]==name]
    seq = seq["model_output"]
    seq = pd.DataFrame(output["model_output"].to_list(), columns=["a1","a2","a3","a4","a5","a6","a7","a8","a9","a10","a11","a12"])
    
    seq = pd.concat([pd.DataFrame(0, index=seq.index, columns=range(0, 256)), seq], axis=1, ignore_index=True)
    
    if len(seq) > 4:
        seq = seq[:4]

    sos = pd.DataFrame([[10]*268])
    
    eos = pd.DataFrame([[20]*268])

    seq = pd.concat([sos, seq, eos], ignore_index=True)
    
    while len(seq) < 6:
           seq = pd.concat([seq, pd.DataFrame([[-100]*268])], ignore_index=True)
           
    #seq = seq.apply(lambda x: np.array(x).astype(np.float32))
    #tokens = []
    #for l in seq:
    #    tokens.append(torch.tensor(l))

    list.append(torch.tensor(seq.values))
outTensor = torch.stack(list)
print(torch.isnan(outTensor))
print(outTensor.shape)


In [ ]:

class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len):
        super().__init__()
        # Modified version from: https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        # max_len determines how far the position can have an effect on a token (window)
        
        # Info
        self.dropout = nn.Dropout(dropout_p)
        
        # Encoding - From formula
        pos_encoding = torch.zeros(max_len, dim_model)
        positions_list = torch.arange(0, max_len, dtype=torch.float).view(-1, 1) # 0, 1, 2, 3, 4, 5
        division_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0)) / dim_model) # 1000^(2i/dim_model)
        
        # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_encoding[:, 0::2] = torch.sin(positions_list * division_term)
        
        # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_encoding[:, 1::2] = torch.cos(positions_list * division_term)
        
        # Saving buffer (same as parameter without gradients needed)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        self.register_buffer("pos_encoding",pos_encoding)
        
    def forward(self, token_embedding: torch.tensor) -> torch.tensor:
        # Residual connection + pos encoding
        return self.dropout(token_embedding + self.pos_encoding[:token_embedding.size(0), :])

In [ ]:
class TransformerEmbedded(nn.Module):
    """
    Model from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    # Constructor
    def __init__(
        self,
        num_tokens,
        dim_model,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dropout_p,
    ):
        super().__init__()

        # INFO
        self.model_type = "TransformerEmbedded"
        self.dim_model = dim_model

        # LAYERS
        self.positional_encoder = PositionalEncoding(
            dim_model=dim_model, dropout_p=dropout_p, max_len=5000
        )
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(
            d_model=dim_model,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dropout=dropout_p,
            batch_first=True
        )
        self.out = nn.Linear(dim_model, num_tokens)
        
    def forward(self, src, tgt, tgt_mask=None, src_pad_mask=None, tgt_pad_mask=None):
        # Src size must be (batch_size, src sequence length)
        # Tgt size must be (batch_size, tgt sequence length)

        # Embedding + positional encoding - Out size = (batch_size, sequence length, dim_model)
        src = self.positional_encoder(src)
        tgt = self.positional_encoder(tgt)
        
        # We could use the parameter batch_first=True, but our KDL version doesn't support it yet, so we permute
        # to obtain size (sequence length, batch_size, dim_model),
        #src = src.permute(1,0,2)
        #tgt = tgt.permute(1,0,2)

        # Transformer blocks - Out size = (sequence length, batch_size, num_tokens)
        transformer_out = self.transformer(src, tgt, tgt_mask=tgt_mask, src_key_padding_mask=src_pad_mask, tgt_key_padding_mask=tgt_pad_mask)
        #print(transformer_out)
        out = self.out(transformer_out)
        
        return out
      
    def get_tgt_mask(self, size) -> torch.tensor:
        # Generates a squeare matrix where the each row allows one word more to be seen
        mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
        mask = mask.float()
        mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
        mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0
        
        # EX for size=5:
        # [[0., -inf, -inf, -inf, -inf],
        #  [0.,   0., -inf, -inf, -inf],
        #  [0.,   0.,   0., -inf, -inf],
        #  [0.,   0.,   0.,   0., -inf],
        #  [0.,   0.,   0.,   0.,   0.]]
        
        return mask

    def create_pad_mask(self, input : torch.tensor, pad_token: int) -> torch.tensor:
        # If matrix = [1,2,3,0,0,0] where pad_token=0, the result mask is
        # [False, False, False, True, True, True]

        t = []
        for seq in input:
            list =[]
            for token in seq:
                b = False
                for value in token:
                    if value != pad_token:
                        b = True
                list.append(b)
            t.append(list)
        return torch.tensor(t)
        
                

In [ ]:
model = TransformerEmbedded(
    num_tokens=268, dim_model=268, num_heads=12, num_encoder_layers=12, num_decoder_layers=12, dropout_p=0.1
)
opt = torch.optim.SGD(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

In [ ]:

def train_loop(model, opt, loss_fn, dataloader):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    model.train()
    total_loss = 0
    
    
    for batch in dataloader:
        X, y = batch[0], batch[1]
        #X, y = torch.tensor(X), torch.tensor(y)

        # Now we shift the tgt by one so with the <SOS> we predict the token at pos 1
        y_input = y[:,:-1]
        y_expected = y[:,1:]
        
        # Get mask to mask out the next words
        sequence_length = y.size(1)
        tgt_mask = model.get_tgt_mask(sequence_length)
        
        #print(y.view(y.size(0), -1).shape)
        pad_mask = model.create_pad_mask(y, pad_token=-100)
        #print(pad_mask)

        # Standard training except we pass in y_input and tgt_mask
        #print(X.isnan().any(),y.isnan().any())
        print(X.dtype, y.dtype)
        pred = model(X, y, tgt_mask, pad_mask)
        print(pred.isnan().any())

        # Permute pred to have batch size first again
        #pred = pred.permute(1, 2, 0)      
        loss = loss_fn(pred, y)

        opt.zero_grad()
        loss.backward()
        opt.step()
    
        total_loss += loss.detach().item()
        
    return total_loss / len(dataloader)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train = TensorDataset(inpTensor.float(), outTensor.float())
batch_size = 20 # Set your desired batch size
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)  # For input data

In [ ]:
def fit(model, opt, loss_fn, train_dataloader, epochs):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    
    # Used for plotting later on
    train_loss_list, validation_loss_list = [], []
    
    print("Training and validating model")
    for epoch in range(epochs):
        print("-"*25, f"Epoch {epoch + 1}","-"*25)
        
        train_loss = train_loop(model, opt, loss_fn, train_dataloader)
        train_loss_list += [train_loss]
        
        #validation_loss = validation_loop(model, loss_fn, val_dataloader)
        #validation_loss_list += [validation_loss]
        
        print(f"Training loss: {train_loss:.4f}")
        #print(f"Validation loss: {validation_loss:.4f}")
        print()
        
    return train_loss_list, validation_loss_list
    
train_loss_list, validation_loss_list = fit(model, opt, loss_fn, train_dataloader, 10)

In [ ]:
model

In [ ]:
def predict(model, input_sequence, max_length=268):
    """
    Method from "A detailed guide to Pytorch's nn.Transformer() module.", by
    Daniel Melchor: https://medium.com/@danielmelchor/a-detailed-guide-to-pytorchs-nn-transformer-module-c80afbc9ffb1
    """
    model.eval()
    
    y_input = input_sequence[:,:,:1]


    for _ in range(max_length):
        # Get source mask
        tgt_mask = model.get_tgt_mask(y_input.size(2))
        
        print(input_sequence.shape, y_input.shape)
        pred = model(input_sequence, y_input)
        
        next_item = pred.topk(1)[1].view(-1)[-1].item() # num with highest probabilits

        # Concatenate previous input with predicted best word
        #print(torch.tensor([[[next_item]]]).shape)
        y_input = torch.cat((torch.tensor([[[next_item]]]), y_input), dim=0)

    return y_input.view(-1).tolist()

In [ ]:
#print(test.shape)
predict(model, test)